In [1]:
import re
import json
import spacy
import spacy_transformers
from tqdm import tqdm
#from LOCAL_SETTINGS import SPACY_MODEL_PATH as spaCy_Model
spaCy_Model = "models/ner_pipeline_22_dec_trf/model-best"


ModuleNotFoundError: No module named 'spacy_transformers'

In [ ]:
def sentencizer(text):


    delimiters_pattern = r'[.|·]'
    sentences = re.split(delimiters_pattern, text)
    #print("sentence: ", sentences)
    return sentences
def clean_text(text):

    text = text.replace("{", "").replace("}", "")


    apostrophes = [' ̓', "᾿", "᾽", "'", "’", "‘"]  # all possible apostrophes
    for apostrophe in apostrophes:
        text = text.replace(apostrophe, "ʼ")
    clean = ' '.join(text.replace('-\n', '').replace('\r', ' ').replace('\n', ' ').split())
    #clean = text.replace('-\n', "").replace('\r', " ").replace('\n', " ")
    print("clean sentence: ",clean)
    return clean
def write_to_jsonl(data_list, file_path="output.jsonl"):
    with open(file_path, 'w', encoding='utf-8') as file:
        for data_dict in data_list:
            json.dump(data_dict, file, ensure_ascii=False)
            file.write('\n')


def create_text_tagging_object(sentences):
    nlp = spacy.load(spaCy_Model)

    sentences_tagged = []
    for sentence in tqdm(sentences, desc="Processing sentences", unit="sentence"):
        # Tokenization and part-of-speech tagging
        doc = nlp(sentence)

        doc_dict = {
            "text": sentence,
            "tokens": [
                {
                    "text": token.text,
                    "lemma": token.lemma_,
                    "pos": token.pos_,
                    "tag": token.tag_,
                    "category": token.ent_type_,
                    # "dep": token.dep_,
                    # "is_alpha": token.is_alpha,
                    # "is_stop": token.is_stop,
                }
                for token in doc
            ]
        }
        sentences_tagged += [doc_dict]
    return sentences_tagged

def read_jsonl_to_list(file_path):
    with open(file_path, 'r') as file:
        data_list = [json.loads(line) for line in file]
    return data_list
def create_data():
    galenus_text = ''
    with open("../assets/texts/TLG0057_galen.txt", 'r') as file:
        galenus_text = file.read()
    galenus_text = clean_text(galenus_text)
    galenus_sentences = sentencizer(galenus_text)
    tagged = create_text_tagging_object(galenus_sentences)
    write_to_jsonl(tagged, "galenus_tagged_data.jsonl")
if __name__ == "__main__":
    create_data()

In [3]:
"""Code to reproduce experiment."""
import timeit

import numpy as np
import spacy
import sqlalchemy
import torch.multiprocessing as mp

from bluesearch.sql import retrieve_sentences_from_sentence_ids

mp.set_start_method('fork')  # for n_process is bigger than 1 

# 1. Initiate spacy model
model = "en_core_web_trf"          # or "en_core_web_lg"
pipes_to_disable = []              # or ['tagger', 'parser', 'attribute_ruler', 'lemmatizer']
entity_ruler = True                # or False
DB_URL = ""                        # URL to the database

spacy.require_gpu()                # or spacy.require_cpu()
nlp = spacy.load(model, disable=pipes_to_disable)
if entity_ruler:
    ruler = nlp.add_pipe("entity_ruler")
    ruler.add_patterns([{"label": "LOCATION", "pattern": "Berlin"}])

# 2. First experiment: nlp(sentence)
# 2.a. Create sentence to process
n_times = 1
sentence = "Emmanuel Macron is living in Paris." * n_times
# 2.b. Results
doc = nlp(sentence)
print(doc.ents)
# 2.b. Compute runtime
time_results = timeit.repeat(lambda: nlp(sentence), repeat=10, number=10)
mean_time = np.mean(time_results)
print(mean_time)

# 3. Second experiment: nlp.pipe(texts)
# 3.a. Load sentences from database
engine = sqlalchemy.create_engine(DB_URL)
n_sentences = 10
n_process = 1 
sentences = retrieve_sentences_from_sentence_ids(np.arange(1, n_sentences+1), engine)
texts = [(sent["text"], {"article_id": sent["article_id"], "sentence_id": sent["sentence_id"]}) for i, sent in sentences.iterrows()]
all_texts.append(texts)
# 3.b. Compute runtime
time_results = timeit.repeat(lambda: list(nlp.pipe(texts, as_tuples=True)), repeat=10, number=10)
mean_time = np.mean(time_results)
print(mean_time)

ModuleNotFoundError: No module named 'sqlalchemy'